In [8]:
import gc
import os
import pickle
import random
import shutil

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import losses, optimizers, metrics, callbacks

In [9]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [10]:
SEED = 123
N_CLASS = 12
MAX_EPOCHS = 200

In [11]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Load data

In [12]:
train_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/train",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/val",
    batch_size=512,
    output_sequence_length=16000
)

test_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/test",
    batch_size=512,
    output_sequence_length=16000)

label_names = np.array(train_ds.class_names)
print("label names:", label_names)

Found 45586 files belonging to 12 classes.
Found 6513 files belonging to 12 classes.
Found 13024 files belonging to 12 classes.
label names: ['down' 'go' 'left' 'no' 'off' 'on' 'right' 'silence' 'stop' 'unknown'
 'up' 'yes']


In [13]:
def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels

train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)
test_ds = test_ds.map(squeeze, tf.data.AUTOTUNE)

## CNN + LSTM Simple

In [14]:
from SpeechModels import get_melspec_model
import tensorflow.keras.layers as L
from keras.models import Model

def CNN_LSTM(n_out, input_length, kernel_size):

    mel_spec_model = get_melspec_model(input_length)
    inputs, outputs = mel_spec_model.inputs, mel_spec_model.outputs
    o_shape = outputs[0].shape
    y1 = L.Conv1D(32, kernel_size, activation='relu', padding='same', input_shape=o_shape[1:])(outputs[0])

    x = L.Bidirectional(L.LSTM(128, return_sequences=False))(y1)  # [b_s, seq_len, vec_dim]

    output = L.Dense(n_out, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model


model = CNN_LSTM(N_CLASS, 16000, 5)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 16000)]      0           []                               
                                                                                                  
 tf.signal.stft (TFOpLambda)    (None, 125, 513)     0           ['input[0][0]']                  
                                                                                                  
 tf.math.abs (TFOpLambda)       (None, 125, 513)     0           ['tf.signal.stft[0][0]']         
                                                                                                  
 tf.tensordot (TFOpLambda)      (None, 125, 80)      0           ['tf.math.abs[0][0]']            
                                                                                              

In [15]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
)
checkpoint = callbacks.ModelCheckpoint(
    filepath="simple_cnn.h5",
    monitor="val_sparse_categorical_accuracy",
    verbose=0,
    save_best_only=True,
    save_weights_only=True
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_sparse_categorical_accuracy', factor=0.5, patience=3, min_lr=0.00001, verbose=1)

In [16]:
history = model.fit(
    train_ds,
    epochs=2,
    validation_data=val_ds,
    shuffle=True,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

Epoch 1/2
90/90 [==============================] - 21s 113ms/step - loss: 1.4907 - sparse_categorical_accuracy: 0.6263 - sparse_categorical_crossentropy: 1.4907 - val_loss: 1.4373 - val_sparse_categorical_accuracy: 0.6258 - val_sparse_categorical_crossentropy: 1.4373 - lr: 0.0010
Epoch 2/2
90/90 [==============================] - 13s 132ms/step - loss: 1.2017 - sparse_categorical_accuracy: 0.6498 - sparse_categorical_crossentropy: 1.2017 - val_loss: 1.0147 - val_sparse_categorical_accuracy: 0.6754 - val_sparse_categorical_crossentropy: 1.0147 - lr: 0.0010


## Experiments

Training will be repeated 5 times with different weights initialization.

In [17]:
EXPERIMENT_NAME = "cnn_lstm_simple"
if os.path.exists(EXPERIMENT_NAME):
    shutil.rmtree(EXPERIMENT_NAME)
    os.mkdir(EXPERIMENT_NAME)
else:
    os.mkdir(EXPERIMENT_NAME)

In [18]:
TRAINING_SEEDS = list(range(5))
results = []
for seed in TRAINING_SEEDS:
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    model = CNN_LSTM(N_CLASS, 16000, 5)


    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.01),
        loss=losses.SparseCategoricalCrossentropy(),
        metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
    )

    history = model.fit(
        train_ds,
        epochs=MAX_EPOCHS,
        validation_data=val_ds,
        shuffle=True,
        callbacks=[early_stopping, reduce_lr]
    )

    with open(os.path.join(EXPERIMENT_NAME, f"history_{seed}.pkl"), "wb") as file:
        pickle.dump(history.history, file)

    eval_results = model.evaluate(test_ds)

    predictions = model.predict(test_ds)
    with open(os.path.join(EXPERIMENT_NAME, f"predictions_{seed}.pkl"), "wb") as file:
        pickle.dump(predictions, file)

    results += [{
        'seed': seed,
        'results': dict(zip(model.metrics_names, eval_results))
    }]
    gc.collect()

results = pd.DataFrame(results)
results = pd.concat([results.drop(["results"], axis=1), results["results"].apply(pd.Series)], axis=1)
results.to_csv(os.path.join(EXPERIMENT_NAME, 'results.csv'))

Epoch 1/200
90/90 [==============================] - 16s 137ms/step - loss: 1.4761 - sparse_categorical_accuracy: 0.6255 - sparse_categorical_crossentropy: 1.4761 - val_loss: 1.3038 - val_sparse_categorical_accuracy: 0.6412 - val_sparse_categorical_crossentropy: 1.3038 - lr: 0.0100
Epoch 2/200
90/90 [==============================] - 13s 137ms/step - loss: 1.0846 - sparse_categorical_accuracy: 0.6791 - sparse_categorical_crossentropy: 1.0846 - val_loss: 0.8415 - val_sparse_categorical_accuracy: 0.7335 - val_sparse_categorical_crossentropy: 0.8415 - lr: 0.0100
Epoch 3/200
90/90 [==============================] - 13s 138ms/step - loss: 0.7086 - sparse_categorical_accuracy: 0.7771 - sparse_categorical_crossentropy: 0.7086 - val_loss: 0.6487 - val_sparse_categorical_accuracy: 0.7866 - val_sparse_categorical_crossentropy: 0.6487 - lr: 0.0100
Epoch 4/200
90/90 [==============================] - 13s 136ms/step - loss: 0.5677 - sparse_categorical_accuracy: 0.8238 - sparse_categorical_crossentr

In [19]:
results

,seed,loss,sparse_categorical_accuracy,sparse_categorical_crossentropy
0,0,0.184072,0.946791,0.184072
1,1,0.234346,0.931511,0.234346
2,2,0.201025,0.943105,0.201025
3,3,0.182946,0.945178,0.182946
4,4,0.208616,0.938959,0.208616
